In [1]:
import pysindy as ps
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib.animation as animation

from scipy.integrate import solve_ivp

%matplotlib inline

## Generating data

In [2]:
L1 = L2 = 4
M1 = M2 = 2
G = 9.8
simulation_duration = 2

dt = .01
rk_integrator_args = {}
rk_integrator_args['rtol'] = 1e-13
rk_integrator_args['method'] = 'RK45'
rk_integrator_args['atol'] = 1e-10

In [3]:
def pendulum_motion(t, input):
    theta_1, theta_2, omega_1, omega_2 = input
    
    d_theta_1 = omega_1
    d_theta_2 = omega_2
    
    d_omega_1 = (-G * (2*M1 + M2)*np.sin(theta_1) - M2 * G * np.sin(theta_1 - 2 * theta_2) - 2*np.sin(theta_1-theta_2)*M2*(omega_2**2 * L2 + omega_1**2 * L1*np.cos(theta_1-theta_2)))/(L1 * (2* M1 + M2 - M2*np.cos(2*theta_1 - 2*theta_2)))
    d_omega_2 = (2*np.sin(theta_1 - theta_2)*(omega_1**2*L1*(M1+M2) + G*(M1+M2)*np.cos(theta_1) + omega_2*L2*M2*np.cos(theta_1 - theta_2)))/(L2 * (2* M1 + M2 - M2*np.cos(2*theta_1 - 2*theta_2)))
    
    d_f = np.array([d_theta_1, d_theta_2, d_omega_1, d_omega_2])
        
    return d_f

In [4]:
def compute_joints_position(thetas_over_time: np.ndarray):
    thetas_1, thetas_2 = thetas_over_time
    x_1,y_1 = np.array([[L1 * np.sin(theta), -L1 * np.cos(theta)] for theta in thetas_1]).T
    x_2,y_2 = np.array([(x + L2*np.sin(theta), y - L2*np.cos(theta)) for theta,x,y in zip(thetas_2, x_1, y_1)]).T
    
    return np.array([x_1, y_1, x_2, y_2])

In [5]:
initial_pendulum_config = np.array([np.deg2rad(80),np.deg2rad(-30),0,0])

t_train = np.arange(0, simulation_duration, dt)
t_train_span = (t_train[0], t_train[-1])
x_train = solve_ivp(pendulum_motion, t_train_span, initial_pendulum_config, t_eval=t_train, **rk_integrator_args).y

x_train_theta = x_train[:2]
x_train_omega = x_train[2:]

train_joints_over_time = compute_joints_position(x_train_theta)

In [6]:
test_initial_pendulum_config = np.array([np.deg2rad(65),np.deg2rad(-50),0,0])
t_test = np.arange(0, simulation_duration, dt)
t_test_span = (t_test[0], t_test[-1])
x_test = solve_ivp(pendulum_motion, t_test_span, test_initial_pendulum_config, t_eval=t_test, **rk_integrator_args).y

x_test_theta = x_test[:2]
x_test_omega = x_test[2:]

test_joints_over_time = compute_joints_position(x_test_theta)

In [7]:
def animate_pendulum_double(data1, data2, filename='double_pendulum_versus.mp4', interval=50, fps=20):
    x1_1, y1_1, x2_1, y2_1 = data1
    x1_2, y1_2, x2_2, y2_2 = data2
    n_frames = min(data1.shape[1], data2.shape[1])  # Make sure they match

    fig, ax = plt.subplots()
    all_x = [x1_1, x2_1, x1_2, x2_2]
    all_y = [y1_1, y2_1, y1_2, y2_2]
    ax.set_xlim(min(map(np.min, all_x)) - 1, max(map(np.max, all_x)) + 1)
    ax.set_ylim(min(map(np.min, all_y)) - 1, max(map(np.max, all_y)) + 1)
    ax.set_aspect('equal')
    ax.set_title('Double Pendulums Side by Side')

    line1, = ax.plot([], [], 'o-', lw=2, label="True pendulum", color='blue')
    line2, = ax.plot([], [], 'o-', lw=2, label="Estimated pendulum", color='red')
    ax.legend()

    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        return line1, line2

    def update(i):
        # Pendulum 1
        x_vals1 = [0, x1_1[i], x2_1[i]]
        y_vals1 = [0, y1_1[i], y2_1[i]]
        line1.set_data(x_vals1, y_vals1)

        # Pendulum 2
        x_vals2 = [0, x1_2[i], x2_2[i]]
        y_vals2 = [0, y1_2[i], y2_2[i]]
        line2.set_data(x_vals2, y_vals2)

        return line1, line2

    ani = animation.FuncAnimation(fig, update, frames=n_frames,
                                  init_func=init, blit=True, interval=interval)

    writer = animation.FFMpegWriter(fps=fps)
    ani.save(filename, writer=writer)
    plt.close(fig)


In [8]:
def draw_state_diagrams(true_thetas, estimated_thetas, plot_file_name):
    fig, ax = plt.subplots(1, 1, sharex=True, figsize=(15, 7))
    
    true_theta_1, true_theta_2 = true_thetas
    estimate_theta_1, estimate_theta_2 = estimated_thetas
    
    ax.plot(true_theta_1, true_theta_2, 'o-', lw=2, label="True pendulum", color='blue')
    ax.plot(estimate_theta_1, estimate_theta_2, 'o-', lw=2, label="Estimated pendulum", color='red')
    plt.title("Joint angles over time")
    plt.xlabel("Theta 1")
    plt.ylabel("Theta 2")
    plt.legend()
    
    plt.savefig(plot_file_name, dpi=1300)
    plt.close()
    

In [9]:
def loop_around_angles(theta: np.ndarray):
    return (np.abs(theta) % np.pi) * np.sign(theta)

In [10]:
def animate_pendulum(data, filename='double_pendulum.mp4', interval=50, fps=20):
    x1, y1, x2, y2 = data
    n_frames = data.shape[1]

    fig, ax = plt.subplots()
    ax.set_xlim(np.min([x1, x2]) - 1, np.max([x1, x2]) + 1)
    ax.set_ylim(np.min([y1, y2]) - 1, np.max([y1, y2]) + 1)
    ax.set_aspect('equal')
    ax.set_title('Double Pendulum Animation')

    line, = ax.plot([], [], 'o-', lw=2)

    def init():
        line.set_data([], [])
        return line,

    def update(i):
        this_x = [0, x1[i], x2[i]]
        this_y = [0, y1[i], y2[i]]
        line.set_data(this_x, this_y)
        return line,

    ani = animation.FuncAnimation(fig, update, frames=n_frames,
                                  init_func=init, blit=True, interval=interval)

    # Save animation
    writer = animation.FFMpegWriter(fps=fps)
    ani.save(filename, writer=writer)
    plt.close(fig)


In [11]:
# animate_pendulum(train_joints_over_time, interval=0.005,fps=60, filename="train_pendulum.mp4")

# Fitting SINDy

In [12]:
standard_trigon_lib = ps.feature_library.FourierLibrary(n_frequencies=2)
standard_polynom_lib = ps.feature_library.PolynomialLibrary(degree=2,include_interaction=True, include_bias=False)
powers = [i for i in range(1,3)]

def compute_trigon_power(t_arg, t_func, power):
    return np.nan_to_num(np.power(1 / t_func(t_arg), power), nan=0, neginf=0, posinf=0)

def compute_power(arg, power):
    return np.nan_to_num(np.power(1 / arg, power), nan=0, neginf=0, posinf=0)

inverse_trigon = [
    *[lambda x: compute_trigon_power(x, np.sin, power) for power in powers], 
    *[lambda x,y: compute_trigon_power(x*y, np.sin, power) for power in powers],
    *[lambda x,y: compute_trigon_power(x/y, np.sin, power) for power in powers],
    *[lambda x,y: compute_trigon_power(x + y, np.sin, power) for power in powers],
    *[lambda x,y: compute_trigon_power(x - y, np.sin, power) for power in powers],
    
    *[lambda x: compute_trigon_power(x, np.cos, power) for power in powers], 
    *[lambda x,y: compute_trigon_power(x*y, np.cos, power) for power in powers],
    *[lambda x,y: compute_trigon_power(x/y, np.cos, power) for power in powers],
    *[lambda x,y: compute_trigon_power(x + y, np.cos, power) for power in powers],
    *[lambda x,y: compute_trigon_power(x - y, np.cos, power) for power in powers],
]

inverse_trigon_names = [
    *[lambda x: f"(1/sin({x}))^{power}" for power in powers], 
    *[lambda x,y: f"(1/sin({x}*{y}))^{power}" for power in powers],
    *[lambda x,y: f"(1/sin({x}/{y}))^{power}" for power in powers],
    *[lambda x,y: f"(1/sin({x}+{y}))^{power}" for power in powers],
    *[lambda x,y: f"(1/sin({x}-{y}))^{power}" for power in powers],
    
    *[lambda x: f"(1/cos({x}))^{power}" for power in powers], 
    *[lambda x,y: f"(1/cos({x}*{y}))^{power}" for power in powers],
    *[lambda x,y: f"(1/cos({x}/{y}))^{power}" for power in powers],
    *[lambda x,y: f"(1/cos({x}+{y}))^{power}" for power in powers],
    *[lambda x,y: f"(1/cos({x}-{y}))^{power}" for power in powers],
]

inverse_polynom = [
    *[lambda x: compute_power(x, power) for power in powers], 
    *[lambda x,y: compute_power(x*y, power) for power in powers],
    *[lambda x,y: compute_power(x/y, power) for power in powers],
    *[lambda x,y: compute_power(x+y, power) for power in powers],
    *[lambda x,y: compute_power(x-y, power) for power in powers]
]

inverse_polynom_names = [
    *[lambda x: f"(1/{x})^{power}" for power in powers], 
    *[lambda x,y: f"(1/{x}*{y})^{power}" for power in powers],
    *[lambda x,y: f"(1/{x}/{y})^{power}" for power in powers],
    *[lambda x,y: f"(1/{x}+{y})^{power}" for power in powers],
    *[lambda x,y: f"(1/{x}-{y})^{power}" for power in powers]
]

custom_trigon_lib = ps.feature_library.CustomLibrary(library_functions=inverse_trigon, function_names=inverse_trigon_names, interaction_only=False)
custom_polynom_lib = ps.feature_library.CustomLibrary(library_functions=inverse_polynom, function_names=inverse_polynom_names, interaction_only=False)

# standard_lib =  standard_trigon_lib + standard_polynom_lib
# all_libs =  custom_trigon_lib * standard_lib + custom_polynom_lib * standard_lib + standard_lib + standard_trigon_lib * standard_polynom_lib  
all_libs =  standard_trigon_lib + standard_polynom_lib + standard_trigon_lib * standard_polynom_lib 

## Differentiating on-the-fly

In [13]:
model = ps.SINDy(feature_library=all_libs,optimizer=ps.optimizers.STLSQ(threshold=0.2e-1, max_iter=35, alpha=0.018, verbose=True), feature_names=['theta_1', 'theta_2', 'omega_1', 'omega_2'])
model.fit(np.nan_to_num(x_train, posinf=0, neginf=0, nan=0, copy=False).T, t=dt)
model.print()

 Iteration ... |y - Xw|^2 ...  a * |w|_2 ...      |w|_0 ... Total error: |y - Xw|^2 + a * |w|_2
         0 ... 1.7163e+02 ... 6.7030e-02 ...        335 ... 1.7169e+02
         1 ... 3.2767e+00 ... 7.2139e-02 ...        311 ... 3.3488e+00
         2 ... 6.0532e-01 ... 7.2693e-02 ...        305 ... 6.7802e-01
         3 ... 8.0502e-01 ... 7.3767e-02 ...        303 ... 8.7878e-01
         4 ... 9.8389e-03 ... 7.3841e-02 ...        303 ... 8.3680e-02
(theta_1)' = 0.034 sin(1 omega_1) + 0.006 sin(2 omega_1) + 1.048 omega_1 + -0.021 sin(1 theta_1) theta_1 omega_1 + -0.106 cos(1 theta_1) omega_1 + -0.019 cos(1 theta_1) omega_1^2 + 0.002 cos(1 theta_2) omega_1 + 0.002 cos(1 theta_2) omega_1^2 + -0.025 sin(1 omega_1) theta_1^2 + 0.004 sin(1 omega_1) omega_1^2 + 0.001 sin(2 theta_2) theta_1 omega_1
(theta_2)' = -0.016 sin(1 omega_2) + 1.023 omega_2 + 0.017 sin(1 theta_1) theta_1 omega_2 + -0.006 sin(1 theta_1) omega_2^2 + -0.008 cos(1 theta_2) omega_2 + -0.012 sin(1 omega_2) theta_1^2 + -0.001 s

In [14]:
odeint_integrator_args = {}
odeint_integrator_args['rtol'] = 1e-13
odeint_integrator_args['atol'] = 1e-10
odeint_integrator_args['full_output'] = 1

blind_simulation, info = model.simulate(test_initial_pendulum_config, t_test, integrator="odeint", integrator_kws=odeint_integrator_args)
blind_simulation = blind_simulation.T
blind_simulation[:2] = loop_around_angles(blind_simulation[:2])


joints_over_time_blind = compute_joints_position(blind_simulation[:2])

In [15]:
animate_pendulum_double(test_joints_over_time,joints_over_time_blind,"blind_pendulum.mp4", interval=0.003, fps=30)
draw_state_diagrams(x_test_theta, blind_simulation[:2], "blind_simulation.png")

## Fitting with known derivatives


In [16]:
true_derivatives = []
for i in range(len(t_train)):
    true_derivatives.append(pendulum_motion(t_train[i], x_train.T[i]))
    
true_derivatives=np.array(true_derivatives)
print(true_derivatives.shape)

(200, 4)


In [17]:
model = ps.SINDy(feature_library=all_libs,optimizer=ps.optimizers.STLSQ(threshold=0.2e-1, max_iter=35, alpha=0.015, verbose=True),  feature_names=['theta_1', 'theta_2', 'omega_1', 'omega_2'])
model.fit(np.nan_to_num(x_train, posinf=0, neginf=0, nan=0, copy=False).T, t=dt,x_dot=true_derivatives, unbias=True)
model.print()

 Iteration ... |y - Xw|^2 ...  a * |w|_2 ...      |w|_0 ... Total error: |y - Xw|^2 + a * |w|_2
         0 ... 1.6673e+02 ... 6.0528e-02 ...        336 ... 1.6679e+02
         1 ... 2.7161e+00 ... 6.4775e-02 ...        310 ... 2.7809e+00
         2 ... 5.3171e-01 ... 6.5234e-02 ...        304 ... 5.9694e-01
         3 ... 4.4970e-01 ... 6.6156e-02 ...        301 ... 5.1586e-01
         4 ... 1.0406e-02 ... 6.6674e-02 ...        300 ... 7.7080e-02
         5 ... 9.9581e-03 ... 6.6680e-02 ...        300 ... 7.6638e-02
(theta_1)' = 1.000 omega_1
(theta_2)' = 1.000 omega_2
(omega_1)' = 3.835 sin(1 theta_1) + -1.777 sin(1 omega_2) + -0.691 cos(1 omega_2) + 2.581 cos(2 theta_2) + -1.030 cos(2 omega_1) + -1.828 sin(2 omega_2) + -0.322 cos(2 omega_2) + 4.140 theta_1 + -1.446 omega_2 + 5.549 theta_1 omega_2 + 0.483 omega_2^2 + 4.881 sin(1 theta_1) omega_2 + 0.823 sin(1 theta_1) theta_1^2 + -0.836 sin(1 theta_1) theta_1 theta_2 + 0.757 sin(1 theta_1) theta_1 omega_2 + -1.037 sin(1 theta_1) omega

In [18]:
informed_simulation, info = model.simulate(test_initial_pendulum_config, t_test, integrator="odeint", integrator_kws=odeint_integrator_args)
informed_simulation = informed_simulation.T
informed_simulation[:2] = loop_around_angles(informed_simulation[:2])

joints_over_time_informed = compute_joints_position(informed_simulation[:2])

/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/pysindy.py:901: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  return odeint(rhs, x0, t, tfirst=True, **integrator_kws)


In [19]:
animate_pendulum_double(test_joints_over_time,joints_over_time_informed,"informed_pendulum.mp4", interval=0.003, fps=30)
draw_state_diagrams(x_test_theta, informed_simulation[:2], "informed_simulation.png")

## Fitting with noise

In [20]:
noise_magnitude = 0.01
train_noise = np.random.normal(0, 1, x_train.shape) * noise_magnitude

### Clean derivatives

In [21]:
model = ps.SINDy(feature_library=all_libs,optimizer=ps.optimizers.STLSQ(threshold=0.33e-1, max_iter=35, alpha=0.015, verbose=True),  feature_names=['theta_1', 'theta_2', 'omega_1', 'omega_2'])
model.fit(np.nan_to_num((x_train + train_noise), posinf=0, neginf=0, nan=0, copy=False).T, t=dt,x_dot = true_derivatives)
model.print()

 Iteration ... |y - Xw|^2 ...  a * |w|_2 ...      |w|_0 ... Total error: |y - Xw|^2 + a * |w|_2
         0 ... 3.7702e+02 ... 2.4291e-01 ...        393 ... 3.7726e+02
         1 ... 4.0144e+00 ... 2.5940e-01 ...        376 ... 4.2738e+00
         2 ... 4.2523e+00 ... 2.5971e-01 ...        372 ... 4.5120e+00
         3 ... 2.4845e+00 ... 2.5972e-01 ...        369 ... 2.7442e+00
         4 ... 2.6252e+00 ... 2.5987e-01 ...        368 ... 2.8850e+00
         5 ... 2.4366e+00 ... 2.5993e-01 ...        368 ... 2.6965e+00
(theta_1)' = -0.058 cos(1 theta_1) + 1739.585 sin(1 omega_1) + 14.158 sin(2 omega_1) + -1049.792 omega_1 + -191.106 omega_1^2 + -0.358 cos(1 theta_1) omega_1^2 + -0.108 cos(1 theta_2) omega_1^2 + 276.114 sin(1 omega_1) omega_1 + -51.861 sin(1 omega_1) omega_1^2 + -717.926 cos(1 omega_1) omega_1 + -85.606 cos(1 omega_1) omega_1^2 + 0.217 sin(2 theta_1) theta_1 omega_1 + 0.147 sin(2 omega_1) theta_1^2
(theta_2)' = 1.000 omega_2
(omega_1)' = -6159354.466 sin(1 theta_1) + -7021

In [22]:
informed_noisy_simulation, info = model.simulate(test_initial_pendulum_config, t_test, integrator="odeint", integrator_kws=odeint_integrator_args)
informed_noisy_simulation = informed_noisy_simulation.T
informed_noisy_simulation[:2] = loop_around_angles(informed_noisy_simulation[:2])

joints_over_time_noisy_informed = compute_joints_position(informed_noisy_simulation[:2])

/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/pysindy.py:901: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  return odeint(rhs, x0, t, tfirst=True, **integrator_kws)


In [23]:
animate_pendulum_double(test_joints_over_time,joints_over_time_noisy_informed,"informed_noisy_pendulum.mp4", interval=0.003, fps=30)
draw_state_diagrams(x_test_theta, informed_noisy_simulation[:2], "informed_noisy_simulation.png")

### Applying smoother

In [83]:
best_t = -1
best_score = 0
best_alpha = 0
best_model = None

for t in np.linspace(1e-3, 2, 40):
    for alpha in np.linspace(0.005, 0.85, 50):
        model = ps.SINDy(feature_library=all_libs,optimizer=ps.optimizers.STLSQ(threshold=t, max_iter=35, alpha=alpha, verbose=False),  feature_names=['theta_1', 'theta_2', 'omega_1', 'omega_2'], differentiation_method=ps.differentiation.SmoothedFiniteDifference(smoother_kws={'window_length': 6, 'polyorder':1,'mode':'nearest'}))
        model.fit(np.nan_to_num((x_train + train_noise), posinf=0, neginf=0, nan=0, copy=False).T, t=dt)
        score = model.score(x_test.T, t=dt, x_dot=true_derivatives)
        
        if(score > best_score):
            best_score = score
            best_t = t
            best_model = model
            best_alpha = alpha
        
print(f"Best threshold: {best_t}")
print(f"Best score: {best_score}")
print(f"Best alpha: {best_alpha}")
best_model.print()

/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.15476923076923077) and eliminated all coefficients
  warnings.warn(
/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.15476923076923077) and eliminated all coefficients
  warnings.warn(
/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.15476923076923077) and eliminated all coefficients
  warnings.warn(
/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.15476923

Best threshold: 0.15476923076923077
Best score: 0.8025993360358707
Best alpha: 0.17744897959183673
(theta_1)' = 1.105 sin(2 theta_1) theta_1 omega_1 + 0.439 sin(2 omega_1) omega_1 omega_2
(theta_2)' = 0.189 cos(2 theta_2) omega_1^2 + 0.261 sin(2 omega_1) omega_1^2 + 0.745 sin(2 omega_2) theta_1^2
(omega_1)' = -7.002 sin(1 theta_2) omega_2^2 + -0.642 cos(1 omega_1) theta_1 omega_2 + 0.506 sin(2 theta_1) omega_2 + -0.737 sin(2 theta_1) omega_1^2 + -1.358 cos(2 theta_1) theta_1 omega_2 + 0.971 cos(2 theta_1) omega_2^2 + -1.019 sin(2 theta_2) omega_1 + 0.937 sin(2 theta_2) theta_1^2 + -0.407 sin(2 theta_2) omega_1^2 + 4.502 sin(2 theta_2) omega_2^2 + -1.293 cos(2 theta_2) omega_2^2 + -0.350 cos(2 omega_1) theta_1 + 0.473 cos(2 omega_1) omega_2^2 + 0.690 cos(2 omega_2) theta_1 omega_1 + -1.053 cos(2 omega_2) theta_1 omega_2 + 1.277 cos(2 omega_2) theta_2 omega_1 + 0.116 cos(2 omega_2) theta_2 omega_2
(omega_2)' = 0.599 sin(2 theta_1) + 0.007 cos(1 omega_1) omega_1^2 + -0.851 cos(1 omega_2) 

/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (2.0) and eliminated all coefficients
  warnings.warn(
/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (2.0) and eliminated all coefficients
  warnings.warn(
/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (2.0) and eliminated all coefficients
  warnings.warn(


In [84]:
smoothed_noisy_simulation, info = best_model.simulate(test_initial_pendulum_config, t_test, integrator="odeint", integrator_kws=odeint_integrator_args)
smoothed_noisy_simulation = smoothed_noisy_simulation.T
smoothed_noisy_simulation[:2] = loop_around_angles(smoothed_noisy_simulation[:2])

joints_over_time_noisy_smmothed = compute_joints_position(smoothed_noisy_simulation[:2])

/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/pysindy.py:901: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  return odeint(rhs, x0, t, tfirst=True, **integrator_kws)


In [85]:
animate_pendulum_double(test_joints_over_time,joints_over_time_noisy_smmothed, "smoothed_noisy_pendulum.mp4", interval=0.003, fps=30)
draw_state_diagrams(x_test_theta, smoothed_noisy_simulation[:2], "smoothed_noisy_simulation.png")